In [107]:
import pandas as pd 
import os
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import numpy as np
import glob
import tensorflow as tf

In [90]:
dataDir=Path('./tsu_ai_m2_dataset/')
annFile = Path('./tsu_ai_m2_dataset/result.json')
coco = COCO(annFile)
imgIds = coco.getImgIds()
imgs = coco.loadImgs(imgIds)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [160]:
imgs

[{'width': 1280,
  'height': 1028,
  'id': 0,
  'file_name': 'images/9/a00c4a27-frame_0.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 1,
  'file_name': 'images/9/97190354-frame_3.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 2,
  'file_name': 'images/9/a4bcfdf2-frame_6.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 3,
  'file_name': 'images/9/df0f6559-frame_9.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 4,
  'file_name': 'images/9/4b8f1c1b-frame_12.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 5,
  'file_name': 'images/9/37f25bf3-frame_15.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 6,
  'file_name': 'images/9/8d2559ae-frame_18.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 7,
  'file_name': 'images/9/5a82c75d-frame_21.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 8,
  'file_name': 'images/9/7c965d52-frame_24.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 9,
  'file_name': 'images/9/75138140-frame_27.jpg'},
 {'width': 1280,
  'height': 1028,
  'id': 1

In [135]:
for img in imgs:
    annIds = coco.getAnnIds(imgIds=[img['id']])
    anns = coco.loadAnns(annIds)
    if len(anns) > 0:
        mask = coco.annToMask(anns[0])
        for i in range(len(anns)):
            mask += coco.annToMask(anns[i])
        im = Image.fromarray((mask * 255).astype(np.uint8), 'L')
        im.save('./tsu_ai_m2_dataset/masks/' + str(img['id']) + '.png')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [101]:
CLASSES = 2

COLORS = ['black', 'yellow']

SAMPLE_SIZE = (1280, 1028)

OUTPUT_SIZE = (1280, 1028)

In [102]:
def load_images(image, mask):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, OUTPUT_SIZE)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image / 255.0
    
    mask = tf.io.read_file(mask)
    mask = tf.io.decode_png(mask)
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.image.resize(mask, OUTPUT_SIZE)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    
    masks = []
    
    for i in range(CLASSES):
        masks.append(tf.where(tf.equal(mask, float(i)), 1.0, 0.0))
    
    masks = tf.stack(masks, axis=2)
    masks = tf.reshape(masks, OUTPUT_SIZE + (CLASSES,))

    return image, masks

def augmentate_images(image, masks):   
    random_crop = tf.random.uniform((), 0.3, 1)
    image = tf.image.central_crop(image, random_crop)
    masks = tf.image.central_crop(masks, random_crop)
    
    random_flip = tf.random.uniform((), 0, 1)    
    if random_flip >= 0.5:
        image = tf.image.flip_left_right(image)
        masks = tf.image.flip_left_right(masks)
    
    image = tf.image.resize(image, SAMPLE_SIZE)
    masks = tf.image.resize(masks, SAMPLE_SIZE)
    
    return image, masks

In [154]:
# images = sorted(glob.glob('tsu_ai_m2_dataset/images/9/*.jpg'))[:284]
images = []
for img in imgs:
    
masks = sorted(glob.glob('tsu_ai_m2_dataset/masks/*.png'))

images_dataset = tf.data.Dataset.from_tensor_slices(images)
masks_dataset = tf.data.Dataset.from_tensor_slices(masks)

dataset = tf.data.Dataset.zip((images_dataset, masks_dataset))

# dataset = dataset.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
# dataset = dataset.repeat(10)
# dataset = dataset.map(augmentate_images, num_parallel_calls=tf.data.AUTOTUNE)

In [157]:
annIds

[1459]

In [156]:
list(dataset)

[(<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\000302ad-frame_177.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/masks\\0.png'>),
 (<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\01a71d52-frame_234.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/masks\\1.png'>),
 (<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\01eb8eb1-frame_333.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/masks\\10.png'>),
 (<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\0292e709-frame_435.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/masks\\100.png'>),
 (<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\04a4fa91-frame_666.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/masks\\101.png'>),
 (<tf.Tensor: shape=(), dtype=string, numpy=b'tsu_ai_m2_dataset/images/9\\05b51134-frame_9

In [114]:
images_and_masks = list(dataset.take(5))

fig, ax = plt.subplots(nrows = 2, ncols = 5, figsize=(15, 5), dpi=125)

for i, (image, masks) in enumerate(images_and_masks):
    ax[0, i].set_title('Image')
    ax[0, i].set_axis_off()
    ax[0, i].imshow(image)
        
    ax[1, i].set_title('Mask')
    ax[1, i].set_axis_off()    
    ax[1, i].imshow(image/1.5)
   
    for channel in range(CLASSES):
        contours = measure.find_contours(np.array(masks[:,:,channel]))
        for contour in contours:
            ax[1, i].plot(contour[:, 1], contour[:, 0], linewidth=1, color=COLORS[channel])

plt.show()
plt.close()

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1315840,1], In[1]: [3,1]
	 [[{{node rgb_to_grayscale/Tensordot/MatMul}}]] [Op:IteratorGetNext]